In [1]:
# This notebook processes and stores a dataset with
# actor/actress names, titles, and ratings

import pandas as pd
import random
import numpy as np
from ast import literal_eval

In [2]:
#Read ratings dataset
ratings = pd.read_table('https://datasets.imdbws.com/title.ratings.tsv.gz', sep='\t')

In [3]:
#Get names for each titleid
names = pd.read_table('https://datasets.imdbws.com/name.basics.tsv.gz', sep='\t')

In [4]:
#Filter to only actors and actresses
namesAct = names[names.primaryProfession.str.contains('actor|actress', na=False)]

In [5]:
#Make titles known for into list
titlesKnown = namesAct.knownForTitles.copy().str.split(pat=',')

In [6]:
#Creates actor and titles dataframe
nameTitles = pd.concat([titlesKnown, namesAct.nconst],
          ignore_index=True, axis=1)
nameTitles.rename(columns={0:'tconst', 1:'nameID'}, inplace=True)

In [7]:
#Read title basics dataset
titles = pd.read_table("https://datasets.imdbws.com/title.basics.tsv.gz", sep="\t")

In [8]:
movies = titles[titles.titleType == 'movie'].copy()

In [9]:
deleteCols = ['originalTitle',
              'isAdult',
              'startYear',
              'endYear',
              'runtimeMinutes',
              'genres',
              'titleType']
movies.drop(deleteCols, axis=1, inplace=True)

In [10]:
# Some actors don't have 4 'famous for' movies
def extract_movies(x, pos):
    if pos >= len(x):
        pass
    else:
        return x[pos]

#Append nameIDs onto each tconst in "movies"
firstTitle = nameTitles['tconst'].apply(extract_movies, pos=0)
secondTitle = nameTitles['tconst'].apply(extract_movies, pos=1)
thirdTitle = nameTitles['tconst'].apply(extract_movies, pos=2)
fourthTitle = nameTitles['tconst'].apply(extract_movies, pos=3)

In [11]:
# Connect titles with nameIDs
# theres probably a more efficient way to do everything below

firstTitleName = pd.concat([firstTitle, nameTitles['nameID']], axis=1)
secondTitleName = pd.concat([secondTitle, nameTitles['nameID']], axis=1)
thirdTitleName = pd.concat([thirdTitle, nameTitles['nameID']], axis=1)
fourthTitleName = pd.concat([fourthTitle, nameTitles['nameID']], axis=1)

In [12]:
# Populate movies with actors for each one
# Preserve movies with left merge

nameTitleFirst = movies.merge(firstTitleName, on='tconst', how='left')

In [13]:
allMoviesNames = nameTitleFirst.groupby('tconst').nameID.apply(list).reset_index()

In [14]:
secondTitleNameGrp = secondTitleName.groupby(['tconst']).nameID.apply(list).reset_index()
thirdTitleNameGrp = thirdTitleName.groupby(['tconst']).nameID.apply(list).reset_index()
fourthTitleNameGrp = fourthTitleName.groupby(['tconst']).nameID.apply(list).reset_index()

In [15]:
# Merges in names, skips names who aren't in movies (who are in tv shows, short films, etc.)

test = allMoviesNames.merge(secondTitleNameGrp, on='tconst', how='left')
test2 = test.merge(thirdTitleNameGrp, on='tconst', how='left')
allMoviesNames = test2.merge(fourthTitleNameGrp, on='tconst', how='left')

In [16]:
# Rename columns and merge into one
allMoviesNames.columns = ['tconst', 'nameID1', 'nameID2', 'nameID3', 'nameID4']
allMoviesNames['names'] = pd.Series(allMoviesNames[['nameID1', 'nameID2', 'nameID3', 'nameID4']].values.tolist())
allMoviesNames.drop(['nameID1', 'nameID2', 'nameID3', 'nameID4'], axis=1, inplace=True)

In [17]:
# Merge ratings with movies and names
ratings.drop(columns=['numVotes'], inplace=True)

In [18]:
allMoviesNamesRatings = allMoviesNames.merge(ratings, on='tconst', how='left')

In [19]:
# Remove movies without ratings
moviesNamesRatings = allMoviesNamesRatings[allMoviesNamesRatings.averageRating.notnull()]

In [20]:
#Simplify array in 'names'
def simplify(x):
    x = repr(x)
    comSep = x.replace('[', '').replace(']', '').replace('\'', '')
    return comSep

test = moviesNamesRatings.names.apply(simplify)